In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import os

In [ ]:
BERKELEY_TRAIN_PATH = "../data/BSDS300/images/train"
BERKELEY_TRAIN_IMG_PATHS = [os.path.join(BERKELEY_TRAIN_PATH, f) for f in os.listdir(BERKELEY_TRAIN_PATH) if os.path.isfile(os.path.join(BERKELEY_TRAIN_PATH, f))]

In [ ]:
def load_img(img_path, scale_factor: int = 8, target_color_space=cv2.COLOR_BGR2LAB):
    """
    Input: image path and downscale factor
    Output: downscaled rgb image, flattened image in target color space, H, W
    """
    img_bgr = cv2.imread(img_path)
    H, W = img_bgr.shape[:2]
    
    # Convert to rgb
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    
    # Downscale image
    img_bgr = cv2.resize(img_bgr, (W // scale_factor, H // scale_factor))
    H, W = img_bgr.shape[:2]
    
    # Convert image to target
    img_target = cv2.cvtColor(img_bgr, target_color_space)
    
    print(f"Loaded image {img_path} with final size ({H}, {W})")
    return img_rgb, img_target.reshape(-1, 3), H, W

In [ ]:
def rgb_gaussian_similarity(img, sigma=250, diag_value=0):
    dist = scipy.spatial.distance_matrix(img, img)
    A = np.exp(-dist**2/sigma**2)
    np.fill_diagonal(A, diag_value)
    return A

def extract_clusters(X, population_threshold=1e-6):
    """
    Extracts the clusters based on the provided threshold and returns the selected RELATIVE ids
    """
    return np.nonzero(X > population_threshold)

In [ ]:
def best_response_update(A, X, t):
    """
    Input: similarity matrix A, strategy distribution X, time step t 
    Output: new distribution of strategies X_{t+1} according to best response dynamics
    """
    # Compute payoff
    payoff = A@X
    best_id = np.argmax(payoff)
    best_vector = np.zeros(len(X))
    best_vector[best_id] = 1.
    # Update population according to best response dynamics
    return (best_vector - X) / (t + 1) + X

In [ ]:
def cluster_game(img, algorithm=best_response_update, similarity=rgb_gaussian_similarity, max_segments=10, max_iter=1000):
    segment_trajectories = []
    cluster_indices = []
    max_segments = 10

    # Initialize color array (with all pixels)
    C = img.copy()
    # Initialize array of indices to keep track of pixels as we prune things from segments to segments
    PID = np.arange(len(C))

    for segment in range(max_segments):
        # Set the new number of objects for this segment
        n = len(C)
        if n == 0:
            break
        # Initialize population and similarity matrix
        X = 1. / n *  np.ones(n)
#         print(C.shape)
        A = similarity(C)

        trajectories = [X]

        # TODO: stop iteration if we reach a threshold
        for t in range(1, max_iter+1):
            # Update population according to best response dynamics
            X = algorithm(A, X, t)
            # Store the current state in the trajectory
            trajectories.append(X.copy())

        # Collect clusters   
        new_clusters = extract_clusters(X)
        cluster_indices.append(PID[new_clusters])
        # Update feature and id matrices
        C = np.delete(C, new_clusters, axis=0)
        PID = np.delete(PID, new_clusters)

        # Store segment trajectory
        segment_trajectories.append(trajectories)

    return cluster_indices, segment_trajectories

In [ ]:
def compute_average_colors(img, cluster_indices):
    # For each clusters, compute the average color
    avg_img = np.zeros_like(img)
    for cluster in cluster_indices:
        avg_img[cluster] = np.average(img[cluster], axis=0)
    return avg_img

def compute_support_img(img_target, cluster_indices):
    support = np.zeros(len(img_lab))
    for i_cluster, cluster in enumerate(cluster_indices):
        support[cluster] = i_cluster + 1
    return support

## Best response dynamics

In [ ]:
trajectories = np.array(segment_trajectories[0])
plt.figure()
for i in range(trajectories.shape[1]):
    plt.plot(np.arange(0, n_iter + 1), trajectories[:, i])
ax = plt.gca()
ax.set_ylim([0., 0.1])

In [ ]:
img_rgb, img_target, H, W = load_img(BERKELEY_TRAIN_IMG_PATHS[3])
cluster_indices, segment_trajectories = cluster_game(img_target)

In [ ]:
support = compute_support_img(img_target, cluster_indices)
support = support.reshape(H, W)

avg_img = compute_average_colors(img_target, cluster_indices)
avg_img = avg_img.reshape(H, W, 3)
avg_img = cv2.cvtColor(avg_img, cv2.COLOR_LAB2RGB)

fig, axs = plt.subplots(1, 3, figsize=(10, 5))
axs[0].imshow(img_rgb)
axs[1].imshow(support)
axs[2].imshow(avg_img)

In [ ]:
for img_path in BERKELEY_TRAIN_IMG_PATHS:
    img_rgb, img_target, H, W = load_img(img_path)
    cluster_indices, segment_trajectories = cluster_game(img_target)
    
    support = compute_support_img(img_target, cluster_indices)
    support = support.reshape(H, W)

    avg_img = compute_average_colors(img_target, cluster_indices)
    avg_img = avg_img.reshape(H, W, 3)
    avg_img = cv2.cvtColor(avg_img, cv2.COLOR_LAB2RGB)
    
    plt.figure()
    fig, axs = plt.subplots(1, 3, figsize=(10, 5))
    axs[0].imshow(img_rgb)
    axs[1].imshow(support)
    axs[2].imshow(avg_img)
    plt.show()

## Replicator dynamics

In [ ]:
def replicator_dynamics_update(A, X, t, kappa=0.):
    """
    Input: similarity matrix A, strategy distribution X, time step t 
    Output: new distribution of strategies X_{t+1} according to best response dynamics
    """
    # Compute payoff
    payoff = A@X
    # Expected payoff
    expected_payoff = X.T@payoff
    # Update population according to replicator dynamics
    X_new = X*(payoff + kappa) / (expected_payoff + kappa)
    return X_new

In [ ]:
similarity_rd = lambda x: rgb_gaussian_similarity(x, sigma=250) 
img_rgb, img_target, H, W = load_img(BERKELEY_TRAIN_IMG_PATHS[70])
cluster_indices, segment_trajectories = cluster_game(img_target, algorithm=replicator_dynamics_update, similarity=similarity_rd)

In [ ]:
support = compute_support_img(img_target, cluster_indices)
support = support.reshape(H, W)

avg_img = compute_average_colors(img_target, cluster_indices)
avg_img = avg_img.reshape(H, W, 3)
avg_img = cv2.cvtColor(avg_img, cv2.COLOR_LAB2RGB)

fig, axs = plt.subplots(1, 3, figsize=(10, 5))
axs[0].imshow(img_rgb)
axs[1].imshow(support)
axs[2].imshow(avg_img)

In [ ]:
for img_path in BERKELEY_TRAIN_IMG_PATHS:
    img_rgb, img_target, H, W = load_img(img_path)
    cluster_indices, segment_trajectories = cluster_game(img_target, algorithm=replicator_dynamics_update, similarity=similarity_rd)
    
    support = compute_support_img(img_target, cluster_indices)
    support = support.reshape(H, W)

    avg_img = compute_average_colors(img_target, cluster_indices)
    avg_img = avg_img.reshape(H, W, 3)
    avg_img = cv2.cvtColor(avg_img, cv2.COLOR_LAB2RGB)
    
    plt.figure()
    fig, axs = plt.subplots(1, 3, figsize=(10, 5))
    axs[0].imshow(img_rgb)
    axs[1].imshow(support)
    axs[2].imshow(avg_img)
    plt.show()

## Infection and immunization dynamics

In [ ]:
def replicator_dynamics_update(A, X, t, kappa=0.):
    """
    Input: similarity matrix A, strategy distribution X, time step t 
    Output: new distribution of strategies X_{t+1} according to best response dynamics
    """
    # Compute payoff
    payoff = A@X
    # Expected payoff
    expected_payoff = X.T@payoff
    # Update population according to replicator dynamics
    X_new = X*(payoff + kappa) / (expected_payoff + kappa)
    return X_new